In [5]:
import numpy as np
import pandas as pd
import string
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/icebear/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
dataframe = pd.read_csv('fake-product-review-detection/dataset.csv')
dataframe.head()

,Unnamed: 0,category,rating,label,text_
0,0,Home_and_Kitchen_5,5.0,CG,love well made sturdi comfort i love veri pretti
1,1,Home_and_Kitchen_5,5.0,CG,love great upgrad origin i 've mine coupl year
2,2,Home_and_Kitchen_5,5.0,CG,thi pillow save back i love look feel pillow
3,3,Home_and_Kitchen_5,1.0,CG,miss inform use great product price i
4,4,Home_and_Kitchen_5,5.0,CG,veri nice set good qualiti we set two month


In [10]:
dataframe.drop('Unnamed: 0',axis=1,inplace=True)

In [11]:
dataframe.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,love well made sturdi comfort i love veri pretti
1,Home_and_Kitchen_5,5.0,CG,love great upgrad origin i 've mine coupl year
2,Home_and_Kitchen_5,5.0,CG,thi pillow save back i love look feel pillow
3,Home_and_Kitchen_5,1.0,CG,miss inform use great product price i
4,Home_and_Kitchen_5,5.0,CG,veri nice set good qualiti we set two month


In [12]:
dataframe.dropna(inplace=True)
dataframe['length'] = dataframe['text_'].apply(len) 

dataframe[dataframe['label']=='OR'][['text_','length']].sort_values(by='length',ascending=False).head().iloc[0].text_

"weak on current scienc after see twice i agre much posit five star review out respect read review i 'll repeat everyth i like present i found goofi over ear hairdo facial hair arrang daniel vitali describ `` wild food expert '' distract ugh ditto david wolf extrem goofi wild hairdo on hand jon gabriel describ `` author weight loss expert '' nice groom good present hi stori person transform fellow pound whew becom jock normal weight inspir christian northrup preserv rank one america 's cutest doctor a realli nice look woman present dr. mercola jason vale kri carr alejandro junger fine it disappoint jami oliv popular uk give babi cow growth fluid pas unscientif popular idea milk none present anyth zilch say work doctor t. colin campbel milk bodi bad it good see present take stand sugar they agre evil sugar refin carbohydr with respect dr. northrup `` it 's fat make fat 's sugar '' statement pas muster commun expert recogn evil sugar not mutual exclus recogn proven danger fat particularl

In [27]:
def convertmyTxt(rv):
    np = [c for c in rv if c not in string.punctuation]
    np = ''.join(np)
    return [w for w in np.split() if w.lower() not in stopwords.words('english')]
x_train,x_test,y_train,y_test = train_test_split(dataframe['text_'],dataframe['label'],test_size=0.25)


In [16]:
pip = Pipeline([('how',CountVectorizer(analyzer=convertmyTxt)),
               ('tfidf',TfidfTransformer()),
                ('classifier',RandomForestClassifier())
               ])

In [19]:
pip.fit(x_train,y_train)

,steps,"[('how', ...), ('tfidf', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [20]:
randomForestClassifier = pip.predict(x_test)
randomForestClassifier
print('Accuracy of the model: ',str(np.round(accuracy_score(y_test,randomForestClassifier)*100,2))+ '%')


Accuracy of the model:  84.42%


In [21]:
pip = Pipeline([
    ('how',CountVectorizer(analyzer=convertmyTxt)),
    ('tfidf',TfidfTransformer()),
    ('classifier',SVC())
])

In [22]:
pip.fit(x_train,y_train)

,steps,"[('how', ...), ('tfidf', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [23]:
supportVectorClassifier = pip.predict(x_test)
supportVectorClassifier

array(['CG', 'OR', 'CG', ..., 'OR', 'OR', 'CG'],
      shape=(10108,), dtype=object)

In [24]:
print('accuracy of the model:',str(np.round(accuracy_score(y_test,supportVectorClassifier)*100,2))+ '%')

accuracy of the model: 88.86%


In [30]:
pip = Pipeline([
    ('how',CountVectorizer(stop_words='english',
                            lowercase=True,
                            max_features=20000,
                            ngram_range=(1,2))),
    ('tfidf',TfidfTransformer()),
    ('classifier',LogisticRegression())
])

In [31]:
pip.fit(x_train,y_train)

,steps,"[('how', ...), ('tfidf', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [32]:
logisticRegression = pip.predict(x_test)
logisticRegression
print('accuracy of the model:',str(np.round(accuracy_score(y_test,logisticRegression)*100,2))+'%')

accuracy of the model: 88.9%
